# Part 1: Data cleaning from Wiki

In [ ]:
$ jupyter trust mynotebook.ipynb

In [1]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df = pd.read_html(html, attrs = {'class':'wikitable'})
df = pd.DataFrame(df[0])
df.columns = df.iloc[0]
df = df[1:]
df.head(5)

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [2]:
df = df[df.Borough != 'Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
df["Neighbourhood"] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'],df['Neighbourhood'])
df.head()

,Postal Code,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df.shape

(103, 3)

# Part2 : Latitude and Longitude

In [5]:
!pip install geocoder

In [6]:
import geocoder
df1 = pd.read_csv('Geospatial_Coordinates.csv')

In [7]:
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df1.shape

(103, 3)

In [9]:
df= df1.merge(df, on = 'Postal Code')
df.head(10)

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
5,M1J,43.744734,-79.239476,Scarborough,Scarborough Village
6,M1K,43.727929,-79.262029,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,43.711112,-79.284577,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,43.716316,-79.239476,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,43.692657,-79.264848,Scarborough,"Birch Cliff, Cliffside West"


In [10]:
def get_lati_long(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_lati_long('M3A')

[43.75245000000007, -79.32990999999998]

In [11]:
postal_code = df['Postal Code']
coords = [ get_lati_long(postal_code) for postal_code in postal_code.tolist()]

In [12]:
df_coords = pd.DataFrame(coords,columns = ['Latitude','Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [13]:
df.head(15)

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.81139,-79.19662,Scarborough,"Malvern, Rouge"
1,M1C,43.78574,-79.15875,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.76575,-79.17470,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.76812,-79.21761,Scarborough,Woburn
4,M1H,43.76944,-79.23892,Scarborough,Cedarbrae
5,M1J,43.74446,-79.23117,Scarborough,Scarborough Village
6,M1K,43.72582,-79.26461,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,43.71289,-79.28506,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,43.72360,-79.23496,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,43.69510,-79.26466,Scarborough,"Birch Cliff, Cliffside West"


# PART 3:

In [14]:
!pip install geopy

In [15]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [185]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Geographical coordinate of Toranto, Ontario are latitude = {},longitude = {}.'.format(latitude,longitude))

Geographical coordinate of Toranto, Ontario are latitude = 43.6534817,longitude = -79.3839347.


In [186]:
map_toronto = folium.Map(location = [latitude,longitude],zoom_start = 10)
#add Marker
for lat,lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'],df['Neighbourhood']):
    label = '{},{}'.format(neighborhood,borough)
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(map_toronto)
map_toronto
    

In [187]:
toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop = True)
toronto_data.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M4E,43.67709,-79.29547,East Toronto,The Beaches
1,M4K,43.68375,-79.35512,East Toronto,"The Danforth West, Riverdale"
2,M4L,43.66797,-79.31467,East Toronto,"India Bazaar, The Beaches West"
3,M4M,43.66213,-79.33497,East Toronto,Studio District
4,M4N,43.72843,-79.38713,Central Toronto,Lawrence Park


In [188]:
map_toronto = folium.Map(location = [latitude,longitude],zoom_start = 11)
#add Marker
for lat,lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'],toronto_data['Neighbourhood']):
    label = '{},{}'.format(neighborhood,borough)
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill = True,
    fill_color = '#3186cc',
    fill_opacity = 0.7,
    parse_html = False).add_to(map_toronto)
map_toronto

#### Define foursquare credentials and version

In [189]:
CLIENT_ID = 'XJ0NNGQMZYY0VTKTSCH1GDMDMVSGYU3RILIZMQCKZWS3H1YA'
CLIENT_SECRET = '2PSJRRVCMZX43XBXVBJOV22KQUO2IU5TUB4NYEM5QUSTSCI1' 
VERSION = '20180605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XJ0NNGQMZYY0VTKTSCH1GDMDMVSGYU3RILIZMQCKZWS3H1YA
CLIENT_SECRET:2PSJRRVCMZX43XBXVBJOV22KQUO2IU5TUB4NYEM5QUSTSCI1


##### Exploring Toronto Neighbourhood 

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
toronto_venues = getNearbyVenues(names = toronto_data['Neighbourhood'], latitudes = toronto_data['Latitude'], longitudes = toronto_data['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [30]:
print('{} venues were returned by Foursquare.'.format(toronto_venues.shape[0]))

1738 venues were returned by Foursquare.


In [190]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,60,60,60,60,60,60
"Brockton, Parkdale Village, Exhibition Place",85,85,85,85,85,85
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",77,77,77,77,77,77
Central Bay Street,76,76,76,76,76,76
Christie,11,11,11,11,11,11
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,26,26,26,26,26,26


In [191]:
print('There are {} uniques categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 229 uniques categories


##### Analyse Each Neighbourhood

In [192]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep ="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood']
fixed_columns = [toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [193]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [224]:
toronto_grouped.shape
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0,0.00,0.016667,0.0,0.016667,0.000000,0.000000,0.000000,...,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.016667
1,"Brockton, Parkdale Village, Exhibition Place",0.011765,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.023529,0.011765,...,0.000000,0.011765,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.011765
2,"Business reply mail Processing Centre, South C...",0.000000,0.0,0.02,0.000000,0.0,0.010000,0.000000,0.010000,0.030000,...,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.0,0.0,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.012987,...,0.012987,0.000000,0.012987,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.012987
4,Central Bay Street,0.000000,0.0,0.00,0.000000,0.0,0.013158,0.013158,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013158,0.013158,0.013158,0.0,0.0,0.0,0.000000


##### Top 5 Venues

In [225]:
num_top_venues = 5
for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending = False).reset_index(drop=True).head(num_top_venues))
    
    print('\n')


----Berczy Park----
            venue  freq
0     Coffee Shop  0.08
1     Cheese Shop  0.03
2  Breakfast Spot  0.03
3    Cocktail Bar  0.03
4        Beer Bar  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0     Coffee Shop  0.06
1            Café  0.06
2             Bar  0.06
3      Restaurant  0.05
4  Sandwich Place  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
              venue  freq
0       Coffee Shop  0.10
1             Hotel  0.05
2        Restaurant  0.04
3              Café  0.03
4  Asian Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.06
2                Café  0.06
3   French Restaurant  0.04
4                Park  0.04


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.12
1  Clothing Store  0.05


#### Put in Pandas DataFrame

In [226]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    return row_categories_sorted.index.values[0:num_top_venues]
toronto_grouped.iloc[0,:]

Neighbourhood                    Berczy Park
Accessories Store                          0
Afghan Restaurant                          0
American Restaurant                        0
Antique Shop                       0.0166667
Aquarium                                   0
Art Gallery                        0.0166667
Art Museum                                 0
Arts & Crafts Store                        0
Asian Restaurant                           0
Athletics & Sports                         0
BBQ Joint                                  0
Baby Store                                 0
Bagel Shop                         0.0166667
Bakery                             0.0333333
Bank                                       0
Bar                                        0
Basketball Stadium                         0
Beer Bar                           0.0333333
Beer Store                                 0
Belgian Restaurant                         0
Bike Trail                                 0
Bistro    

In [227]:
num_top_venues = 10
indicators = ['st' , 'nd' , 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th most common venue'.format(ind+1))

neighbourhood_venues_sorted = pd.DataFrame(columns = columns)
neighbourhood_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhood_venues_sorted.iloc[ind,1:] = return_most_common_venues(toronto_grouped.iloc[ind,:], num_top_venues)

neighbourhood_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,Berczy Park,Coffee Shop,Farmers Market,Beer Bar,Breakfast Spot,Cocktail Bar,Restaurant,Cheese Shop,Bakery,Seafood Restaurant,Lounge
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Bar,Café,Restaurant,Gift Shop,Sandwich Place,Nightclub,Japanese Restaurant,Supermarket,Furniture / Home Store
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Restaurant,Café,Italian Restaurant,Bar,Asian Restaurant,Salon / Barbershop,Thai Restaurant,Pub
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Café,Coffee Shop,Bar,Park,French Restaurant,Bakery,Sandwich Place,Restaurant,Gym / Fitness Center
4,Central Bay Street,Coffee Shop,Clothing Store,Restaurant,Sandwich Place,Sushi Restaurant,Café,Plaza,Bubble Tea Shop,Cosmetics Shop,Bookstore


#### Cluster Neighborhoods

In [229]:
kclusters = 10
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood',1)
kmeans = KMeans(n_clusters = kclusters, init = 'k-means++', max_iter = 100, random_state = 0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

##### creating new dataframe  that include the cluster as well as the top 10 neighborhood


In [230]:
neighbourhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data
toronto_merged = toronto_merged.join(neighbourhood_venues_sorted.set_index('Neighbourhood'), on = 'Neighbourhood')

toronto_merged.head()


,Postal Code,Latitude,Longitude,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,M4E,43.67709,-79.29547,East Toronto,The Beaches,4.0,Health Food Store,Pub,Trail,Neighborhood,Yoga Studio,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1,M4K,43.68375,-79.35512,East Toronto,"The Danforth West, Riverdale",9.0,Intersection,Bus Line,Business Service,Coffee Shop,Park,Ice Cream Shop,Grocery Store,Discount Store,Farmers Market,Event Space
2,M4L,43.66797,-79.31467,East Toronto,"India Bazaar, The Beaches West",0.0,Park,Gym,Italian Restaurant,Pet Store,Pizza Place,Pub,Coffee Shop,Restaurant,Movie Theater,Sandwich Place
3,M4M,43.66213,-79.33497,East Toronto,Studio District,0.0,Pizza Place,Diner,Brewery,Italian Restaurant,Arts & Crafts Store,Bar,Bakery,Gastropub,Coffee Shop,Sushi Restaurant
4,M4N,43.72843,-79.38713,Central Toronto,Lawrence Park,1.0,Bus Line,Swim School,Yoga Studio,Elementary School,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [231]:

toronto_merged = toronto_merged.dropna(subset = ['Cluster Labels'])
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,M4E,43.67709,-79.29547,East Toronto,The Beaches,4,Health Food Store,Pub,Trail,Neighborhood,Yoga Studio,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
1,M4K,43.68375,-79.35512,East Toronto,"The Danforth West, Riverdale",9,Intersection,Bus Line,Business Service,Coffee Shop,Park,Ice Cream Shop,Grocery Store,Discount Store,Farmers Market,Event Space
2,M4L,43.66797,-79.31467,East Toronto,"India Bazaar, The Beaches West",0,Park,Gym,Italian Restaurant,Pet Store,Pizza Place,Pub,Coffee Shop,Restaurant,Movie Theater,Sandwich Place
3,M4M,43.66213,-79.33497,East Toronto,Studio District,0,Pizza Place,Diner,Brewery,Italian Restaurant,Arts & Crafts Store,Bar,Bakery,Gastropub,Coffee Shop,Sushi Restaurant
4,M4N,43.72843,-79.38713,Central Toronto,Lawrence Park,1,Bus Line,Swim School,Yoga Studio,Elementary School,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [232]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)
x = np.arange(kclusters)
ys = [i + x + (1*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# THANK YOU!....